<a target="_blank" href="https://colab.research.google.com/github/michalis0/Cloud-and-Advanced-Analytics/blob/main/02.%20BigQuery/week_02_exercises.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Week 2: Google BigQuery

## Walkthrough

### For Colab users

In [1]:
from google.colab import auth

auth.authenticate_user()
print("Authenticated")

Authenticated


#### For Jupyter users ONLY

Make sure to replace "PATH_TO_CREDENTIALS_FILE" with the *absolute* path to the JSON service account key (e.g., "C:/Users/John/credentials.json"). Here is the [documentation](https://cloud.google.com/iam/docs/creating-managing-service-account-keys).

In [ ]:
!pip install google-cloud-bigquery

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "PATH_TO_CREDENTIALS_FILE"

### Connect to BigQuery

In [4]:
import pandas as pd
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(project="bsa-2021-sandbox-001122")  # CHANGE WITH YOUR OWN PROJECT ID

To run the cells below, you must first create a new dataset in BigQuery and upload the data from week 2 (`Teams.csv` and `Players.csv`) as tables. Since the CSV files are well-formed, you can automatically infer the schema when loading the data.

In [6]:
# Create a reference to the World Cup dataset
dataset_ref = client.dataset("world_cup", project="bsa-2021-sandbox-001122")  # CHANGE WITH YOUR OWN DATASET (NOT TABLE) NAME AND PROJECT ID

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [7]:
# List the tables in the dataset
tables = list(client.list_tables(dataset))
for table in tables:  
    print(table.table_id)

Players
Teams


In [8]:
# Create a reference to the "Teams" table
table_ref = dataset_ref.table("Teams")

# API request - fetch the table
teams_table = client.get_table(table_ref)

# Same for the "Players" table
table_ref = dataset_ref.table("Players")
players_table = client.get_table(table_ref)

In [9]:
# Display the schema of the "Teams" table 
players_table.schema

[SchemaField('surname', 'STRING', 'NULLABLE', None, ()),
 SchemaField('team', 'STRING', 'NULLABLE', None, ()),
 SchemaField('position', 'STRING', 'NULLABLE', None, ()),
 SchemaField('minutes', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('shots', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('passes', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('tackles', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('saves', 'INTEGER', 'NULLABLE', None, ())]

In [10]:
# Show a preview of the "Teams" table using pandas
client.list_rows(teams_table, max_results=5).to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Honduras,38,3,0,1,2,0,3,7,0
1,North Korea,105,3,0,0,3,1,12,2,0
2,Greece,13,3,1,0,2,2,5,5,0
3,Cameroon,19,3,0,0,3,2,5,5,0
4,New Zealand,78,3,0,3,0,2,2,6,0


### Example of queries

In [11]:
q1 = """
select *
from bsa-2021-sandbox-001122.world_cup.Teams
where ranking < 20
order by ranking
"""

query_job = client.query(q1)
query_job.to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Brazil,1,5,3,1,1,9,4,7,2
1,Spain,2,6,5,0,1,7,2,3,0
2,Portugal,3,4,1,2,1,7,1,8,1
3,Netherlands,4,6,6,0,0,12,5,15,0
4,Italy,5,3,0,2,1,4,5,5,0
5,Germany,6,6,4,0,2,13,3,8,1
6,Argentina,7,5,4,0,1,10,6,7,0
7,England,8,4,1,2,1,3,5,6,0
8,France,9,3,0,1,2,1,4,6,1
9,Greece,13,3,1,0,2,2,5,5,0


In [12]:
# Join the two tables
q2 = """
select teams.*, players.surname, players.position, players.shots,
players.minutes, players.passes, players.tackles, players.saves
from bsa-2021-sandbox-001122.world_cup.Teams teams
join bsa-2021-sandbox-001122.world_cup.Players players
on teams.team = players.team
"""

query_job = client.query(q2)
query_job.to_dataframe().head(10)

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards,surname,position,shots,minutes,passes,tackles,saves
0,Honduras,38,3,0,1,2,0,3,7,0,Jerry Palacios,forward,0,84,19,0,0
1,Honduras,38,3,0,1,2,0,3,7,0,Walter Martinez,forward,0,125,43,1,0
2,Honduras,38,3,0,1,2,0,3,7,0,Chavez,defender,0,270,62,3,0
3,Honduras,38,3,0,1,2,0,3,7,0,Izaguirre,defender,0,180,51,8,0
4,Honduras,38,3,0,1,2,0,3,7,0,Mauricio Sabilluen,defender,0,90,38,5,0
5,Honduras,38,3,0,1,2,0,3,7,0,Mendoza,defender,0,180,51,3,0
6,Honduras,38,3,0,1,2,0,3,7,0,Valladares,goalkeeper,0,270,51,0,12
7,Honduras,38,3,0,1,2,0,3,7,0,Espinoza,midfielder,0,135,36,4,0
8,Honduras,38,3,0,1,2,0,3,7,0,Guevara,midfielder,0,156,60,6,0
9,Honduras,38,3,0,1,2,0,3,7,0,Thomas,midfielder,0,114,37,5,0


You can try more queries on your own! (or re-use the queries you wrote in the other notebook of week 2 with the SQLite extension)

## Exercises

Let's build some queries for the `accidents_2016` data from the NHTSA traffic facilities dataset, which is publicly available on Google BigQuery.

### Setting up

In [13]:
# Create a reference to the traffic facilities dataset
traffic_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
traffic_dataset = client.get_dataset(traffic_ref)

In [14]:
# Construct a reference to the "accident_2016" table
accidents_ref = traffic_ref.table("accident_2016")

# API request - fetch the table
accidents_table = client.get_table(accidents_ref)

In [15]:
# Display schema
accidents_table.schema

[SchemaField('state_number', 'INTEGER', 'NULLABLE', 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC). For more info on the codes, please look at <C1/V1/D1/PC1/P1/NM1 State Number> section in the pdf: https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812315', ()),
 SchemaField('state_name', 'STRING', 'NULLABLE', 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC).', ()),
 SchemaField('consecutive_number', 'INTEGER', 'NULLABLE', 'This data element is the unique case number assigned to each crash. It appears on each data file and is used to merge information from the data files together. xxxxxx Two Characters for State Code followed by Four Characters for Case Number', ()),
 SchemaField('number_of_vehicle_f

In [16]:
# Show a preview of the "accident_2016" table
client.list_rows(accidents_table, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,county,city,day_of_crash,month_of_crash,year_of_crash,day_of_week,hour_of_crash,minute_of_crash,national_highway_system,land_use,land_use_name,functional_system,functional_system_name,ownership,ownership_name,route_signing,route_signing_name,trafficway_identifier,trafficway_identifier_2,milepoint,latitude,longitude,special_jurisdiction,special_jurisdiction_name,first_harmful_event,first_harmful_event_name,manner_of_collision,manner_of_collision_name,relation_to_junction_within_interchange_area,relation_to_junction_specific_location,relation_to_junction_specific_location_name,type_of_intersection,work_zone,relation_to_trafficway,relation_to_trafficway_name,light_condition,light_condition_name,atmospheric_conditions_1,atmospheric_conditions_1_name,atmospheric_conditions_2,atmospheric_conditions_2_name,atmospheric_conditions,atmospheric_conditions_name,school_bus_related,rail_grade_crossing_identifier,hour_of_notification,minute_of_notification,hour_of_arrival_at_scene,minute_of_arrival_at_scene,hour_of_ems_arrival_at_hospital,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,56,Wyoming,560099,1,1,0,0,0,1,1,7,0,4,12,2016,1,3,45,1,1,Rural,1,Interstate,1,State Highway Agency,1,Interstate,I-80,,2533,41.742200,-106.494300,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,No,1,Non-Junction,Not an Intersection,None,4,On Roadside,2,Dark – Not Lighted,4,Snow,11,Blowing Snow,4,Snow,No,0000000,4,27,4,49,88,88,0,None,0,None,0,None,1,0,2016-12-04 03:45:00+00:00
1,16,Idaho,160226,1,1,0,0,0,3,3,53,0,16,12,2016,6,16,33,1,1,Rural,1,Interstate,1,State Highway Agency,1,Interstate,I-84,,1781,42.609000,-114.356000,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,No,1,Non-Junction,Not an Intersection,None,3,On Median,1,Daylight,10,Cloudy,11,Blowing Snow,11,Blowing Snow,No,0000000,16,35,16,49,99,99,0,None,0,None,0,None,2,0,2016-12-16 16:33:00+00:00
2,5,Arkansas,50006,1,1,0,0,0,2,2,115,3450,6,1,2016,4,3,36,1,2,Urban,1,Interstate,1,State Highway Agency,1,Interstate,I-40-22 RAMP,,839,35.284200,-93.095453,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,Yes,5,Entrance/Exit Ramp Related,Not an Intersection,None,1,On Roadway,2,Dark – Not Lighted,1,Clear,0,No Additional Atmospheric Conditions,1,Clear,No,0000000,99,99,99,99,99,99,20,Police-Pursuit-Involved (Since 1994),0,None,0,None,1,0,2016-01-06 03:36:00+00:00
3,48,Texas,482027,1,1,0,0,0,5,5,201,3280,7,8,2016,1,15,28,1,2,Urban,1,Interstate,1,State Highway Agency,1,Interstate,I-69/EASTEX FWY,,83,29.979078,-95.277997,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,No,1,Non-Junction,Not an Intersection,None,10,Separator,1,Daylight,1,Clear,0,No Additional Atmospheric Conditions,1,Clear,No,0000000,16,3,16,7,16,35,0,None,0,None,0,None,1,0,2016-08-07 15:28:00+00:00
4,12,Florida,121570,1,1,0,0,0,5,5,11,2570,21,7,2016,5,19,37,1,2,Urban,1,Interstate,1,State Highway Agency,1,Interstate,I-95,,99998,26.275011,-80.125189,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,No,1,Non-Junction,Not an Intersection,None,4,On Roadside,1,Daylig

### Questions

You can access the table using `from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`.

1) How many crashes happened in each month of 2016?

In [ ]:
q3 = """
YOUR QUERY HERE
"""

query_job_3 = client.query(q3)
query_job_3.to_dataframe()

2) Let's call the accidents involving more than 5 motor vehichles in transport involved as "massive_crash", and all others as "small crash". Return the number of each type of crash for each weather condition.

**Hint**: Use columns `atmospheric_conditions_1_name` and `number_of_motor_vehicles_in_transport_mvit`.

In [ ]:
q4 = """
YOUR QUERY HERE
"""

query_job_4 = client.query(q4)
query_job_4.to_dataframe()

3) Find the top 5 states with the highest number of drunk drivers involved in accidents.

In [ ]:
q5 = """
YOUR QUERY HERE
"""

query_job_5 = client.query(q5)
query_job_5.to_dataframe()

4) Let's find out the states where using a cell phone is a major cause of accidents. Return the first 5 state names along with the number of crashes where the driver was distracted "while manipulating cellular phone".

**Hint**: You need to join the `distract_2016` and `accident_2016` tables.

In [ ]:
q6 = """
YOUR QUERY HERE
"""

query_job_6 = client.query(q6)
query_job_6.to_dataframe()

5) Sometimes, a "live animal" is a reason that a driver maneuvers and causes an accident. In which months of the year do such accidents happen the most? Return the month and the number of accidents. 

**Hint**: You need to join the `maneuver_2016` and `accident_2016` tables. 

In [ ]:
q7 = """
YOUR QUERY HERE
"""

query_job_7 = client.query(q7)
query_job_7.to_dataframe()

6) In the state of Alaska, in which months do "poor road conditions" cause the most accidents? Return the month and the number of accidents. 

**Hint**: Again, you will need the `maneuver_2016` table. 

In [ ]:
q8 = """
YOUR QUERY HERE
"""

query_job_8 = client.query(q8)
query_job_8.to_dataframe()

7) Sometimes, a factor of the accident is that the vehicle has a problem with the "brake system". Filter the accidents with this factor and return the `driver_maneuvered_to_avoid_name` column along with the related number of accidents.
 
**Hint**: You need to join the `maneuver_2016` and `factor_2016` tables.

In [ ]:
q9 = """
YOUR QUERY HERE
"""

query_job_9 = client.query(q9)
query_job_9.to_dataframe()

8) Sometimes, a non-motorist is involved in a crash. For each state, find the number of crashes in which an "inattentive" person was involved. Return the state name and number of crashes.

**Hint**: You need to join the `nmcrash_2016` and `accident_2016` tables.

In [ ]:
q10 = """
YOUR QUERY HERE
"""

query_job_10 = client.query(q10)
query_job_10.to_dataframe()